In [10]:
import numpy as np
import cv2
import os
import glob as gb
import tensorflow as tf
from numpy import expand_dims
from cv2 import resize,INTER_CUBIC

In [11]:
train_dir='../data/ids'
Train_Data=tf.keras.preprocessing.image.ImageDataGenerator(
    horizontal_flip=True,
    rescale=1/255.0,
).flow_from_directory(train_dir,batch_size=16,subset="training",target_size=(224,224),shuffle=False)

Found 1400 images belonging to 7 classes.


In [12]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import ZeroPadding2D, Convolution2D, MaxPooling2D, Dropout, Flatten, Activation

def vgg_face():
    model = Sequential()
    model.add(ZeroPadding2D((1,1),input_shape=(224,224, 3)))
    model.add(Convolution2D(64, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))
    
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(128, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))
    
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))
    
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))
    
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))
    
    model.add(Convolution2D(4096, (7, 7), activation='relu'))
    model.add(Dropout(0.5))
    model.add(Convolution2D(4096, (1, 1), activation='relu'))
    model.add(Dropout(0.5))
    model.add(Convolution2D(2622, (1, 1)))
    model.add(Flatten())
    model.add(Activation('softmax'))
    return model

In [13]:
model = vgg_face()

model.load_weights('models/vgg_face_weights.h5')

In [14]:
from tensorflow.keras.models import Model
model = Model(inputs=model.layers[0].input, outputs=model.layers[-2].output)
print(model.summary())

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 zero_padding2d_13_input (In  [(None, 224, 224, 3)]    0         
 putLayer)                                                       
                                                                 
 zero_padding2d_13 (ZeroPadd  (None, 226, 226, 3)      0         
 ing2D)                                                          
                                                                 
 conv2d_16 (Conv2D)          (None, 224, 224, 64)      1792      
                                                                 
 zero_padding2d_14 (ZeroPadd  (None, 226, 226, 64)     0         
 ing2D)                                                          
                                                                 
 conv2d_17 (Conv2D)          (None, 224, 224, 64)      36928     
                                                           

                                                                 
 max_pooling2d_7 (MaxPooling  (None, 28, 28, 256)      0         
 2D)                                                             
                                                                 
 zero_padding2d_20 (ZeroPadd  (None, 30, 30, 256)      0         
 ing2D)                                                          
                                                                 
 conv2d_23 (Conv2D)          (None, 28, 28, 512)       1180160   
                                                                 
 zero_padding2d_21 (ZeroPadd  (None, 30, 30, 512)      0         
 ing2D)                                                          
                                                                 
 conv2d_24 (Conv2D)          (None, 28, 28, 512)       2359808   
                                                                 
 zero_padding2d_22 (ZeroPadd  (None, 30, 30, 512)      0         
 ing2D)   

In [15]:
embedding_vector = model.predict(Train_Data,steps=len(Train_Data), verbose=1)
y_train = Train_Data.labels

88/88 [==============================] - 370s 4s/step


In [16]:
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
X_train,X_test,y_train,y_test=train_test_split(embedding_vector,y_train,test_size=0.1,shuffle=True, stratify=y_train,random_state=42)
X_train,y_train = shuffle(X_train,y_train)

In [17]:
# Standarize features
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA


scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

pca = PCA(n_components=128)
X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)

In [18]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

clf = SVC(kernel='linear',C=2.,class_weight='balanced',decision_function_shape='ovo',probability=True)
clf.fit(X_train, y_train)

y_predict = clf.predict(X_test)

print(f'The Accuracy of VGGFace2 is {accuracy_score(y_test,y_predict)*100} %')

The Accuracy of VGGFace2 is 99.28571428571429 %


In [19]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Predict on the test data
y_pred_svm = clf.predict(X_test)

    # Evaluate the classifier
accuracy_svm = accuracy_score(y_test, y_pred_svm)
cm_svm = confusion_matrix(y_test, y_pred_svm)
report_svm = classification_report(y_test, y_pred_svm)

print("SVM Accuracy:", accuracy_svm)
print("SVM Confusion Matrix:\n", cm_svm)
print("SVM Classification Report:\n", report_svm)

SVM Accuracy: 0.9928571428571429
SVM Confusion Matrix:
 [[20  0  0  0  0  0  0]
 [ 0 20  0  0  0  0  0]
 [ 0  0 20  0  0  0  0]
 [ 0  0  0 20  0  0  0]
 [ 0  0  1  0 19  0  0]
 [ 0  0  0  0  0 20  0]
 [ 0  0  0  0  0  0 20]]
SVM Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00        20
           1       1.00      1.00      1.00        20
           2       0.95      1.00      0.98        20
           3       1.00      1.00      1.00        20
           4       1.00      0.95      0.97        20
           5       1.00      1.00      1.00        20
           6       1.00      1.00      1.00        20

    accuracy                           0.99       140
   macro avg       0.99      0.99      0.99       140
weighted avg       0.99      0.99      0.99       140



In [20]:
from sklearn.tree import DecisionTreeClassifier

DTC = DecisionTreeClassifier()
DTC.fit(X_train, y_train)

y_predict_1 = DTC.predict(X_test)
print(f'The Accuracy of VGGFace2 is {accuracy_score(y_test,y_predict_1)*100} %')

The Accuracy of VGGFace2 is 95.71428571428572 %


In [21]:
y_pred_dt = DTC.predict(X_test)

    # Evaluate the classifier
accuracy_dt = accuracy_score(y_test, y_pred_dt)
cm_dt = confusion_matrix(y_test, y_pred_dt)
report_dt = classification_report(y_test, y_pred_dt)

print("Decision Tree Accuracy:", accuracy_dt)
print("Decision Tree Confusion Matrix:\n", cm_dt)
print("Decision Tree Classification Report:\n", report_dt)

Decision Tree Accuracy: 0.9571428571428572
Decision Tree Confusion Matrix:
 [[20  0  0  0  0  0  0]
 [ 0 19  1  0  0  0  0]
 [ 0  0 18  0  2  0  0]
 [ 0  0  0 19  1  0  0]
 [ 0  0  1  0 19  0  0]
 [ 0  0  0  0  0 20  0]
 [ 0  0  0  0  1  0 19]]
Decision Tree Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00        20
           1       1.00      0.95      0.97        20
           2       0.90      0.90      0.90        20
           3       1.00      0.95      0.97        20
           4       0.83      0.95      0.88        20
           5       1.00      1.00      1.00        20
           6       1.00      0.95      0.97        20

    accuracy                           0.96       140
   macro avg       0.96      0.96      0.96       140
weighted avg       0.96      0.96      0.96       140



In [22]:
from sklearn.ensemble import RandomForestClassifier

RFC = RandomForestClassifier()
RFC.fit(X_train, y_train)

y_predict_2 = RFC.predict(X_test)
print(f'The Accuracy of VGGFace2 is {accuracy_score(y_test,y_predict_2)*100} %')

The Accuracy of VGGFace2 is 98.57142857142858 %


In [23]:
y_pred_rf = RFC.predict(X_test)

    # Evaluate the classifier
accuracy_rf = accuracy_score(y_test, y_pred_rf)
cm_rf = confusion_matrix(y_test, y_pred_rf)
report_rf = classification_report(y_test, y_pred_rf)

print("Random Forest Accuracy:", accuracy_rf)
print("Random Forest Confusion Matrix:\n", cm_rf)
print("Random Forest Classification Report:\n", report_rf)

Random Forest Accuracy: 0.9857142857142858
Random Forest Confusion Matrix:
 [[20  0  0  0  0  0  0]
 [ 0 20  0  0  0  0  0]
 [ 0  0 20  0  0  0  0]
 [ 0  0  0 19  1  0  0]
 [ 0  0  1  0 19  0  0]
 [ 0  0  0  0  0 20  0]
 [ 0  0  0  0  0  0 20]]
Random Forest Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00        20
           1       1.00      1.00      1.00        20
           2       0.95      1.00      0.98        20
           3       1.00      0.95      0.97        20
           4       0.95      0.95      0.95        20
           5       1.00      1.00      1.00        20
           6       1.00      1.00      1.00        20

    accuracy                           0.99       140
   macro avg       0.99      0.99      0.99       140
weighted avg       0.99      0.99      0.99       140



In [24]:
from joblib import dump

dump(scaler, 'models/scaler.joblib') 
dump(pca, 'models/pca_model.joblib')
dump(clf, 'models/SVC.joblib') 


['models/SVC.joblib']

In [25]:
model.save("models/model.h5")